# Example Environment Variable & Main Invocation in Jupyter Notebook

## This code example abstracts the need for CLI and provides abstraction sufficient to support cross script/notebook utilization

In [9]:
################################################################
#- Create the environment variables
#- Now these environment variables are in the shell environment as shown below
#-   notice how the MYPROJECT_ naming schema keeps everything together, reduces
#-   confusion and is consistent with authoritative naming schemas and translation
#-   to things like JSON/XML without conversion of concern of mappings nor collision
#-   with other projects or environment variables
################################################################
try:
    __IPYTHON__
except NameError:
    inIPython = False
    print("You're in not in Jypter lab.  Not executing %env as that's a Jupyter specific function.")
    print("You would normally call a shell script that sets the env space and then invokes the script itself.")
    print("See ./014_JupyterPythonMainArgs.sh and ./014_JupyterPythonMainArgs.env which sets the environment and invokes the script.")
    print("If not called and this script is called directly by python expect failure.")
else:
    inIPython = True
    #treat as JSON array structure so you can simulate an array later
    %env MYPROJECT_NETCDF_PRODUCT_DOMAIN=["vis_lmi_c", "chlor_a", "vis_lmi_Kd", "horiz_vis_Z"]
    #decimal degrees only
    %env MYPROJECT_GEOSPATIAL_LAT_COORDS=30.0
    #%env OFM_GRAPHICS_LON=-90.0
    %env MYPROJECT_GEOSPATIAL_LON_COORDS=-87.5
    #in domain of [png, jpg, gif]
    %env MYPROJECT_GRAPHICS_EXTENSION=png
    %env MYPROJECT_GLOBAL_YMD=20200501
    %env MYPROJECT_REGION=MissBight
    %env

env: MYPROJECT_NETCDF_PRODUCT_DOMAIN=["vis_lmi_c", "chlor_a", "vis_lmi_Kd", "horiz_vis_Z"]
env: MYPROJECT_GEOSPATIAL_LAT_COORDS=30.0
env: MYPROJECT_GEOSPATIAL_LON_COORDS=-87.5
env: MYPROJECT_GRAPHICS_EXTENSION=png
env: MYPROJECT_GLOBAL_YMD=20200501
env: MYPROJECT_REGION=MissBight


# Includes

In [10]:
##
## @myproject myproject_geospatial_example
##
##  Read a Product Generator (derivative of BioCast 3D output) NetCDF and calculates specific output graphics for the sponsor with the following base parameters for each forecast period:
##   - vis_lmi_c
##   - vis_lmi_Kd
##   - chlor_a
##
##   -v, --version    prints the version of this software package.
##   -o, --ofile  =  *name of the NetCDF output file (contains product data)
##   -i, --ifile  =  *name of the NetCDF Product Generator input file (BIOCAST derivative)
##   -c, --clobber=   overwrite existing output file
##
##   Copyright (C) 2020 Naval Research Lab (NRL)
##
##  This program is free software: you can redistribute it and/or modify
##  it under the terms of the GNU General Public License as published by
##  the Free Software Foundation, either version 3 of the License, or
##  (at your option) any later version.
##
##  This program is distributed in the hope that it will be useful,
##  but WITHOUT ANY WARRANTY; without even the implied warranty of
##  MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
##  GNU General Public License for more details.
##
##  You should have received a copy of the GNU General Public License
##  along with this program.  If not, see <http://www.gnu.org/licenses/>.

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# INCLUDES
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
import os
import datetime
import sys
import traceback
import gc
import glob
import getopt
import inspect
import math
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import norm
import scipy.ndimage
import warnings
import json

from pathlib import Path

# GLOBAL Variables

In [11]:
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# GLOBAL VARIABLES
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## Define a debug switch used to produce log output that is more details than standard information.
DEBUG = 1

## Define a debug data switch used to produce verbose log output, results in very verbose output.
DEBUG_DATA = 1

## Official name of the application.
VERSION_NAME = "MYPROJECT NetCDF Thing"

## Official acronym of the application.
VERSION_ACRONYM = "MNT"

## _FillValue default used when NetCDF required field is not present / provided.
FILL_VALUE = -32767.0

## Character encoding to ensure special notation is carried into the code and beyond without error.
ENCODING = "utf-8"

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# GLOBAL CONSTANTS
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

##Normally a _FillValue would be used but different NetCDF implementations are treating some values differently or don't even have the concept.
MINIMAL_VALUE = 0.0001

VERSION_MAJOR   = 1
VERSION_MINOR   = 0
VERSION_RELEASE = 0

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# APPLICATION VARIABLES
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
## Product NetCDF, one tau, string name of the file
input_filename = ""

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# GLOBAL CONFIGURATION
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
os.environ["PYTHONIOENCODING"] = ENCODING######################################################
pd.options.display.float_format = '{:,.4f}'.format
np.set_printoptions(precision=4)


# FUNCTIONS (Generic)

In [12]:
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# WARNING / ERROR Management
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

##  Operational construct intended to capture warning generated by the script engine.
#
def fxn():
    warnings.warn("deprecated", DeprecationWarning)


with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# FUNCTIONS
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
## Prints system information regarding libraries this application uses via the debugging framework.
#
#  @return string
#
def lib_diagnostics():
    print("System version    #:{:>12}".format(sys.version))
    try:
        netcdf4_version_info = nc.getlibversion().split(" ")
        print("netCDF4 version   #:{:>12}".format(netcdf4_version_info[0]))
    except Exception as e:
        e;
    try:        
        print("Matplotlib version#:{:>12}".format(matplt.__version__))
    except Exception as e:
        e;
    try:        
        print("Numpy version     #:{:>12}".format(np.__version__))
    except Exception as e:
        e;
    try:    
        print("Pandas version    #:{:>12}".format(pd.__version__))
    except Exception as e:
        e;
    try:       
        print("SciPy version     #:{:>12}".format(sp.__version__))
    except Exception as e:
        e;
    try:        
        print("Xarray version    #:{:>12}".format(xr.__version__))
    except Exception as e:
        e;

    return

##  Returns the verion information (numeric only) for this application.
#  @return String
#
def get_version():

   return str(VERSION_MAJOR) + "." + str(VERSION_MINOR) + "." + str(VERSION_RELEASE)




##  Returns the verion information (full) for this application.
#  @return String
#
def get_full_version():
    resultant = str(VERSION_NAME) + "  v" + str(get_version())
    return resultant


## Returns a string sufficient for NetCDF history updates
#  @return String
#
def get_version_title():

    resultant = str(get_full_version()) + " example program."
    return resultant


## Outputs full version to stdout.
#  @return stdout
#  @see get_full_version
#
def printversion():

    print(get_full_version())
    lib_diagnostics()


## Prints usage information (arguments passed, etc.) to stdout
#  @return stdout
#
def printusage():

    print("")
    printversion()
    print("  -h, --help       prints the version and these instructions.")
    print("  -v, --version    prints the version of this software package.")
    print("  -i, --ifile  =  *name of the NetCDF Product input file (Product Generator generated, all forecasts)."   )
    print("  * - indicates required argument.")
    print(" ")
    print("This program comes with ABSOLUTELY NO WARRANTY.")
    print("This is free software, and you are welcome to redistribute it")
    print("under certain conditions; see the LICENSE file for details.")


# BASIC DEFENSES

In [13]:
## Validates a string against known attack techniques by scanning for specific characters.
#
#  We don't want the following:
#  - `-` at the start of the file name (might be construed as a switch)
#  - `$, &, |, ;, <, >, `, !, *, ", \ `
#
#  @param (string) testsubject - incoming string to test for non-conforming characters.
#  @return int
#
def validstring(inc_value):

    #print("...validstring: {}".format(inc_value))
    #print("......type: {}".format(type(inc_value)))
    
    if (isinstance(inc_value, (list, tuple, int, float))):
        print("...inc_value ({}) is of type ({}) and will not be evaluated.".format(inc_value, type(inc_value)))
        return 1
    elif (isinstance(inc_value,(str))):
        testsubject=str(inc_value)
        if testsubject[0] == "-":
            return 0
        elif "$" in testsubject or "&" in testsubject or "|" in testsubject:
            return 0
        elif ";" in testsubject or "`" in testsubject or "!" in testsubject:
            return 0
        elif "*" in testsubject or '"' in testsubject or "\\" in testsubject:
            return 0
        else:
            return 1
    else:
        print("...inc_value({}) will not be evaluated for input.  Type too complex.".format(inc_value))
        return 1

# FAILURE MANAGEMENT FUNCTION

In [14]:
## Central management of errors, once invoked application fails
#
#  @param (string)    inc_reason    - reason for failure.
#  @param (Exception) inc_exception - Python Exception class passed providing support as to the reason for failure.
#  @return exit
#
def failure_management(inc_reason, inc_exception):
    print("Exception:")
    print("...." + str(inc_exception))
    print(traceback.format_exc())
    print("")
    print(inc_reason)
    printusage()
    exit(1)

# ARGUMENT HARVESTING

In [15]:
def argument_management():

    ARGS={}  #create a dictionary / hashmap
   
    try:
        ARGS["MYPROJECT_NETCDF_PRODUCT_DOMAIN"] = json.loads(os.getenv("MYPROJECT_NETCDF_PRODUCT_DOMAIN"))
    except Exception as e:
        print("...argument_management:")
        print("......exception encountered with var ({}) as follows: {}".format("MYPROJECT_NETCDF_PRODUCT_DOMAIN", e))
        print("......as this is argument parsing execution will not continue until this issue is resolved, check environment variable inputs.")
        exit(1)
    
    #additional checks and protections would be appropriate here.
    ARGS["MYPROJECT_GEOSPATIAL_LAT_COORDS"] = float(os.getenv("MYPROJECT_GEOSPATIAL_LAT_COORDS"))
    ARGS["MYPROJECT_GEOSPATIAL_LON_COORDS"] = float(os.getenv("MYPROJECT_GEOSPATIAL_LON_COORDS"))
    
    ARGS["MYPROJECT_GRAPHICS_EXTENSION"]    = str(os.getenv("MYPROJECT_GRAPHICS_EXTENSION"))
    ARGS["MYPROJECT_GLOBAL_YMD"]            = str(os.getenv("MYPROJECT_GLOBAL_YMD"))
    ARGS["MYPROJECT_REGION"]                = str(os.getenv("MYPROJECT_REGION"))
    
    for da_key in ARGS.keys():
        if not (validstring(ARGS[da_key])) :
            print("ERROR: There was a problem with {} environment variable.  Input validation did not pass due to an erroneous character.  Please inputs your environment variable values if you want continued program execution.".format(da_key))
    
    return ARGS


# MAIN METHOD DECLARATION

In [17]:
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# MAIN
# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
## Main method.
#
#  @param (list) argv - Arguments provided via command-line interface (CLI).
#
def main(argv):

    if DEBUG:
        print("Started " + str(VERSION_NAME) + ".")
        print("")

    ###################################################################################
    # - Argument management
    ###################################################################################
    #try:
    #    opts, args = getopt.getopt(
    #        argv,
    #        "vhO:o:I:i:A:a:S:s:",
    #        ["version", "help", "ifile=", "ofile=", "sfile=", "afile="],
    #    )
    #except getopt.GetoptError:
    #    printusage()
    #    sys.exit(2)
    print("")
    printversion()
    print("")
    ARGS=argument_management()
    
    #process code
    print("")
    print("The {} processes the following variables:".format(get_full_version()))
    for my_value in ARGS["MYPROJECT_NETCDF_PRODUCT_DOMAIN"]:
        print("...{}".format(my_value))
    print("")
    print("And produces output graphics with the a {} extension.".format(ARGS["MYPROJECT_GRAPHICS_EXTENSION"]))
    print("The {} region will be processed for Latitude ({}), Longitude ({}) coordinates.".format(ARGS["MYPROJECT_REGION"], ARGS["MYPROJECT_GEOSPATIAL_LAT_COORDS"], ARGS["MYPROJECT_GEOSPATIAL_LON_COORDS"] ))
    
    if DEBUG:
        print("")
        print("Ended " + str(VERSION_NAME) + ".")
